# Challenge Data Engineer - LATAM

Por: Camilo Gutierrez

En el análisis de tweets en Python, se aborda la optimización tanto de la memoria como del tiempo de ejecución. Para la gestión eficiente de la memoria, se emplean estrategias que evitan cargar el archivo completo en la memoria, haciendo uso de librerías nativas de bajo nivel como json para la lectura línea por línea. Este enfoque minimiza el consumo de recursos. 

Por otro lado, para optimizar el tiempo de ejecución, se recurre a la librería Pandas. Esta herramienta ofrece un rendimiento superior en el procesamiento de grandes conjuntos de datos, permitiendo realizar operaciones de manera más rápida y eficiente. 

Al combinar estrategias específicas para cada objetivo, se logra un análisis integral que maximiza tanto la eficiencia en el manejo de la memoria como en el tiempo de ejecución

Para el manejo de entorno virtuales y dependencias se usa Pipenv que es una herramienta útil para:

* Aislar las dependencias de los proyectos
* Registrar las versiones de las dependencias
* Facilitar el mantenimiento de los proyectos
* Facilitar la colaboración con otros desarrolladores

In [1]:
# Funciones para optimizacion de tiempo y memoria
from q1_memory import q1_memory
from q1_time import q1_time
from q2_memory import q2_memory
from q2_time import q2_time
from q3_memory import q3_memory
from q3_time import q3_time
import time

%load_ext memory_profiler

 Los repositorios de Git pueden volverse muy grandes si se suben archivos grandes, lo que puede hacer que sea más difícil trabajar con el repositorio. Por esta razón el archivo no se sube al repositorio (Se agregó al gitignore). 

In [2]:
file_path = "farmers-protest-tweets-2021-2-4.json"

## Las top 10 fechas donde hay más tweets. 

Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días.

```python
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Obtiene las 10 fechas más activas y su usuario más activo correspondiente de un archivo JSON.

    Args:
        file_path (str): La ruta al archivo JSON.

    Returns:
        List[Tuple[datetime.date, str]]: lista de tuplas que contiene la fecha y el usuario más activo para cada fecha.
    """
    df = pd.read_json(file_path, lines=True)

    # Convertir 'date' a datetime y extraer la fecha
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Extraer el nombre de usuario del diccionario anidado 'user'
    df['username'] = df['user'].apply(lambda row: row['username'])

    # Agrupar por 'date' y contar los tweets, obtener el usuario más activo por fecha
    top_dates_df = df.groupby('date')['username'].agg(
        total_tweets='size',
        most_active_user=lambda x: x.value_counts().idxmax()
    ).nlargest(10, columns='total_tweets')

    # Convierte el resultado a una lista de tuplas
    top_dates_list = top_dates_df[['most_active_user']].to_records().tolist()

    return top_dates_list
```

In [3]:
start_time = time.time()
%memit top_10 = q1_time(file_path)
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")
top_10

peak memory: 2859.55 MiB, increment: 2739.30 MiB
Execution time: 5.1993489265441895 seconds


[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

```python
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Esta función lee un archivo JSON que contiene tweets y devuelve una lista de las 10 fechas con mayor cantidad de tweets,
    junto con el usuario más activo para cada una de esas fechas.

    Args:
        file_path (str): La ruta del archivo JSON que contiene los tweets.

    Returns:
        List[Tuple[datetime.date, str]]: Una lista de tuplas que contiene la fecha y el usuario más activo para cada una de las 10 fechas con mayor cantidad de tweets.
    """
    # Se inicializa los diccionarios para contabilizar la actividad de tweets
    tweet_counts = defaultdict(int)  # Contador de tweets por fecha
    tweets_activity = defaultdict(lambda: defaultdict(int))  # Contador de tweets por fecha y usuario

    # Abrir el archivo JSON y procesar cada línea
    with open(file_path, 'r') as file:
        for line in file:
            # Leer los tweets desde la línea del archivo JSON
            tweet = json.loads(line)
            
            # Extraer la fecha y el nombre de usuario del tweet
            date = datetime.fromisoformat(tweet['date']).date()
            username = tweet['user']['username']

            # Contar la cantidad de tweets por fecha y usuario
            tweet_counts[date] += 1
            tweets_activity[date][username] += 1

    # Obtener las 10 fechas con mayor cantidad de tweets
    top_10_dates = Counter(tweet_counts).most_common(10)

    # Encontrar el usuario más activo para cada una de las 10 fechas con mayor trafico
    top_users = [(date, Counter(tweets_activity[date]).most_common(1)[0][0]) for date, _ in top_10_dates]

    return top_users
```

In [4]:
start_time = time.time()
%memit top_10 = q1_memory(file_path)
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")
top_10

peak memory: 305.50 MiB, increment: 0.23 MiB
Execution time: 2.2031350135803223 seconds


[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Esta aproximación al problema logra una reducción sustancial en el consumo máximo de memoria y también mejora significativamente la velocidad de ejecución. Esto se debe a que estamos manejando un conjunto de datos relativamente pequeño, haciendo que esta estrategia de optimización de memoria sea la solución más eficiente en términos de tiempo de ejecución.

## Los top 10 emojis más usados

Para determinar si un texto contiene un emoji, se puede usar una expresión regular para buscar secuencias de caracteres Unicode que coincidan con el patrón de un emoji usando REGEX por ejemplo. Sin embargo, esto puede ser complejo y propenso a errores.

Por eso se usa la libreria emoji que proporciona funciones integradas para extraer emojis de texto, lo que puede ser más eficiente que hacerlo manualmente. Ademas emoji admite una variedad de formatos de emojis, incluidos los emojis Unicode, los emojis HTML, entre otros.

Nota: Si un tweet tiene un mismo emoji repetido, cada aparición se contará.

```python
def count_emojis(text) -> Counter:
    """
    Cuenta la cantidad de emojis en un texto.

    Args:
        text (str): El texto en el que se contarán los emojis.

    Returns:
        Counter: Un objeto Counter que contiene la cantidad de cada emoji encontrado en el texto.
    """
    # Analiza el texto en busca de emojis y devuelve una lista de objetos EmojiData.
    data = emoji.analyze(text)

    # Extrae los caracteres de cada objeto EmojiMatch y los almacena en una lista.
    emoji_list = [em.chars for em in data]

    # Crea un objeto Counter a partir de la lista de emojis y devuelve el resultado.
    return Counter(emoji_list) if emoji_list else None

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    """
    Calcula los 10 emojis más comunes en un archivo JSON.

    Args:
        file_path (str): La ruta al archivo JSON.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene los emojis más comunes y su frecuencia.
    """
    df = pd.read_json(file_path, lines=True)
    
    # Se unen todo el contenido de los tweets en un solo string
    conteo_de_emojis = count_emojis("".join(df.content.values))
    
    return conteo_de_emojis.most_common(10)
```

In [5]:
start_time = time.time()
%memit top_10_emojis = q2_time(file_path)
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")
top_10_emojis

peak memory: 2971.00 MiB, increment: 2665.50 MiB
Execution time: 11.406476020812988 seconds


[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

```python
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Lee un archivo JSON y cuenta la frecuencia de los emojis en los tweets.

    Args:
        file_path (str): La ruta del archivo JSON.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene los 10 emojis más comunes y su frecuencia.
    """
    # Leer el archivo JSON
    with open(file_path, 'r', encoding='utf-8') as file:
        total_emoji_counter = Counter()
        for line in file:
            tweet = json.loads(line) 
            # Extraer los emojis de cada tweet
            emoji_counter = count_emojis(tweet['content'])
            if emoji_counter:
                # Sumar los emojis de cada tweet al contador total 
                total_emoji_counter += emoji_counter

    # Obtener los 10 emojis más comunes
    top_emojis = total_emoji_counter.most_common(10)

    return top_emojis
```

In [6]:
start_time = time.time()
%memit top_10_emojis = q2_memory(file_path)
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")
top_10_emojis

peak memory: 493.14 MiB, increment: 0.05 MiB
Execution time: 8.347600221633911 seconds


[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

## Top 10 histórico de usuarios más influyentes

Al contar las menciones para determinar los usuarios más influyentes, se tuvo en cuenta que un solo tweet puede mencionar a uno o más usuarios. Cada mención se contó individualmente, independientemente de cuántas veces un usuario específico fue mencionado en un solo tweet. Se usa la columna **mentionedUsers**

```python
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Cuenta el número de veces que cada usuario mencionado aparece en un archivo que contiene tweets.

    Args:
        file_path (str): La ruta al archivo que contiene los tweets.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas, donde cada tupla contiene el nombre de usuario de un usuario mencionado
        y el número de veces que fue mencionado, ordenado en orden descendente según el conteo de menciones.
    """
   # Inicializar un contador para contar menciones de usuarios
    mentioned_users_counter = Counter()

    # Abrir el archivo JSON de tweets y procesar cada línea
    with open(file_path, 'r', encoding='utf-8') as file:
        # Iterar sobre cada línea del archivo
        for line in file:
            # Cargar el tweet desde la línea como un objeto JSON
            tweet = json.loads(line)
            
            # Obtener la lista de usuarios mencionados en el tweet
            mentioned_user = tweet['mentionedUsers']
            
            # Verificar si hay usuarios mencionados en el tweet
            if not mentioned_user:
                continue
            
            # Actualizar el contador para cada usuario mencionado en el tweet
            for user in mentioned_user:
                mentioned_users_counter[user['username']] += 1
    
    # Obtener las 10 menciones más comunes y convertirlas a una lista de tuplas
    conteo_lista = mentioned_users_counter.most_common(10)
    
    # Devolver la lista de tuplas ordenada en orden descendente según el conteo de menciones
    return conteo_lista
```

In [7]:
start_time = time.time()
%memit top_10_mentioned_users = q3_memory(file_path)
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")
top_10_mentioned_users

peak memory: 492.17 MiB, increment: 0.03 MiB
Execution time: 2.184135913848877 seconds


[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

```python
def q3_time(file_path: str) -> List[Tuple[str, int]]:
    """
    Lee un archivo JSON desde la ruta especificada y cuenta cuántas veces se menciona cada usuario.
    
    Args:
        file_path (str): La ruta al archivo JSON.
        
    Returns:
        List[Tuple[str, int]]: Una lista de tuplas que contiene el nombre de usuario y la cantidad de menciones,
                               ordenada en orden descendente según la cantidad.
    """
     # Leer el archivo JSON en un DataFrame
    df = pd.read_json(file_path, lines=True)

     # Seleccionar la columna 'mentionedUsers' y eliminar filas con valores nulos
    df_mentioned_users = df['mentionedUsers'].dropna()

     # Aplicar una función para extraer los nombres de usuario y explotar las listas en filas separadas
    df_mentioned_users = df_mentioned_users.apply(lambda x: [x['username'] for x in x]).explode()

     # Contar la frecuencia de cada nombre de usuario
    conteo = df_mentioned_users.value_counts()

     # Convertir el resultado a un DataFrame, tomar las primeras 10 filas y convertir a lista de tuplas
    conteo_list = conteo.to_frame().head(10).to_records().tolist()
    
     # Devolver la lista de tuplas ordenada en orden descendente según la cantidad de menciones
    return conteo_list
```

In [8]:
start_time = time.time()
%memit top_10_mentioned_users = q3_time(file_path)
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")
top_10_mentioned_users

peak memory: 2980.41 MiB, increment: 2488.23 MiB
Execution time: 5.1369709968566895 seconds


[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]